In [57]:
import numpy as np
import pandas as pd
import re

In [9]:
import os 
os.getcwd()

'/media/sukant/Moz1/DS/Books'

In [11]:
imdb_dir = 'aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
        f = open(os.path.join(dir_name, fname))
        texts.append(f.read())
        f.close()
    if label_type == 'neg':
        labels.append(0)
    else:
        labels.append(1)

In [16]:
len(texts), len(labels)

(12500, 12500)

In [61]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

maxlen = 100
#Cuts off reviews after 100 words
training_samples = 200
#Trains on 200 samples
validation_samples = 10000
#Validates on 10,000 samples
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [54]:
list(map(len, map(lambda x: x.split(), texts[:15])))

[179, 313, 122, 305, 66, 247, 208, 397, 195, 235, 152, 190, 173, 51, 145]

In [51]:
list(map(len,sequences[:15]))

[176, 299, 116, 300, 67, 240, 206, 366, 190, 221, 152, 187, 160, 49, 135]

In [28]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 64421 unique tokens.


In [29]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (12500, 100)
Shape of label tensor: (12500,)


In [88]:
data[1]

array([   1,  696,  595, 1397,  203,    1,  585,   13,    9,  382, 1375,
       3048,    1,  262,    6,   25,    3,   17, 4754,    6,   21,   72,
          5, 2735,   13,  288,   33,    6,  303,    7,    1, 3281,    2,
         32,  147,   22,  990,  555,   45,  311,  161,    6,    3,  190,
         17, 5532,   13,   45, 4959, 5550, 1880, 1938, 2178,    2,   58,
         93, 8807,   14,    3, 9788, 2833,   13, 1397,  180,   43,   15,
       1335,  339,  406,    7,    3,  190,  161,   13,    3,    1,   19,
       1478,    1,  316,    4, 3609,    2,    1,  321,  372,  425,    6,
        279, 1140,   15,   10,   27,  104,   11,   19,  156,    3,  178,
        184], dtype=int32)

In [33]:
data.shape

(12500, 100)

In [30]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [39]:
'''
Splits the data into a training set and a
validation set, but first shuffles the data,
because you’re starting with data in which
samples are ordered (all negative first, then
all positive)
'''
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

In [40]:
x_train.shape

(200, 100)

In [ ]:
glove_dir = '/Users/fchollet/Downloads/glove.6B'
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))